# **SDG Prediction Validation**

## **Dependencies**

In [45]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## **Load Data**

### Load GIZ Project Data As Ground Truth

GIZ Running Projects can be used as ground truth for SDG.

In [46]:
running_giz_df = pd.read_excel("../../src/ground_truths/giz_running_projects.xlsx")

running_giz_df.head(1)

,*Projektnummer,"Projektbezeichnung, deutsch","Projektbezeichnung, englisch",CRS-Schlüssel,CRS-Bezeichnung,CRS-Gewichtung,2. CRS-Schlüssel,2. CRS-Bezeichnung,2. CRS-Gewichtung,3. CRS-Schlüssel,3. CRS-Bezeichnung,3. CRS-Gewichtung,4. CRS-Schlüssel,4. CRS-Bezeichnung,4. CRS-Gewichtung,SDG Hauptziele,"Projektziele, deutsch","Projektziele, englisch",BMZ-Schwerpunkt
0,2008.9076.4,Unterstützung bei der Einführung eines Kühlsch...,Refrigerator recycling system,41010.0,Umweltpolitik und -verwaltung,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13;15;,Unterstützung beim Aufbau eines beispielhaften...,Supporting the introduction of an exemplary ta...,"Umweltpolitik, Schutz und nachhaltige Nutzung ..."


### Load Merged DF

In [47]:
df = pd.read_csv("../../src/merged_orgas.csv")
df.head(1)

,iati_id,iati_orga_id,orga_abbreviation,orga_full_name,title_en,title_other,title_main,organization,country_code,country,...,actual_start,planned_end,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,docs,sgd_pred_code
0,DE-1-199535469,DE-1,bmz,Bundesministerium für wirtschaftliche Zusammen...,Studies and Experts Fund,Studien- und Fachkräftefonds,Studies and Experts Fund,Bundesministerium für wirtschaftliche Zusammen...,['NA'],NA;,...,2024-01-01T00:00:00Z,2026-12-31T00:00:00Z,2024-01-31T00:00:00Z,2024-01-31T00:00:00Z,43010;,Multisector aid;,430;,Other Multisector;,['https://www.giz.de/projektdaten/projects.act...,15


### Extract Project Numbers out of IATI ID

In [48]:
bmz_df = df[df.iati_orga_id == "DE-1"]

bmz_df["pn"] = "NaN"

for index, row in bmz_df.iterrows():
    try:
        pn_raw = row['iati_id'].split("-")[2]
        pn = f"{pn_raw[:4]}.{pn_raw[4:8]}.{pn_raw[8:]}"
        bmz_df.loc[index, "pn"] = pn
    except:
        pass

bmz_df.head(2)

C:\Users\UserNA6153\AppData\Local\Temp\ipykernel_15396\514410831.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bmz_df["pn"] = "NaN"


,iati_id,iati_orga_id,orga_abbreviation,orga_full_name,title_en,title_other,title_main,organization,country_code,country,...,planned_end,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,docs,sgd_pred_code,pn
0,DE-1-199535469,DE-1,bmz,Bundesministerium für wirtschaftliche Zusammen...,Studies and Experts Fund,Studien- und Fachkräftefonds,Studies and Experts Fund,Bundesministerium für wirtschaftliche Zusammen...,['NA'],NA;,...,2026-12-31T00:00:00Z,2024-01-31T00:00:00Z,2024-01-31T00:00:00Z,43010;,Multisector aid;,430;,Other Multisector;,['https://www.giz.de/projektdaten/projects.act...,15,1995.3546.9
1,DE-1-199535469,DE-1,bmz,Bundesministerium für wirtschaftliche Zusammen...,Studies and Experts Fund,Studien- und Fachkräftefonds,Studies and Experts Fund,Bundesministerium für wirtschaftliche Zusammen...,['NA'],NA;,...,2026-12-31T00:00:00Z,2024-01-31T00:00:00Z,2024-01-31T00:00:00Z,43010;,Multisector aid;,430;,Other Multisector;,['https://www.giz.de/projektdaten/projects.act...,15,1995.3546.9


### Extract running GIZ projects just with 1 sdg to achieve better compareability 

In [49]:
running_giz_df = running_giz_df[running_giz_df['SDG Hauptziele'].str.len() == 3]

print(f"{len(running_giz_df)} with 1 SDG")
running_giz_df.head(1)

521 with 1 SDG


,*Projektnummer,"Projektbezeichnung, deutsch","Projektbezeichnung, englisch",CRS-Schlüssel,CRS-Bezeichnung,CRS-Gewichtung,2. CRS-Schlüssel,2. CRS-Bezeichnung,2. CRS-Gewichtung,3. CRS-Schlüssel,3. CRS-Bezeichnung,3. CRS-Gewichtung,4. CRS-Schlüssel,4. CRS-Bezeichnung,4. CRS-Gewichtung,SDG Hauptziele,"Projektziele, deutsch","Projektziele, englisch",BMZ-Schwerpunkt
1,2010.9226.1,GIZ Mgmt - Vertrag 1 - Unterstützung des BMU ...,GIZ Mgmt - Vertrag 1 - Unterstützung des BMU b...,99810.0,Nicht spezifizierte Sektoren (fortgeführt als ...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17;,EU Twinning,EU Twinning,Gestaltungsspielraum


### Match all running giz projects with IATI BMZ data 

In [50]:
# rename project number column in running_giz_df
running_giz_df.rename(columns={'*Projektnummer': 'pn'}, inplace=True)
running_giz_df.head(1)

,pn,"Projektbezeichnung, deutsch","Projektbezeichnung, englisch",CRS-Schlüssel,CRS-Bezeichnung,CRS-Gewichtung,2. CRS-Schlüssel,2. CRS-Bezeichnung,2. CRS-Gewichtung,3. CRS-Schlüssel,3. CRS-Bezeichnung,3. CRS-Gewichtung,4. CRS-Schlüssel,4. CRS-Bezeichnung,4. CRS-Gewichtung,SDG Hauptziele,"Projektziele, deutsch","Projektziele, englisch",BMZ-Schwerpunkt
1,2010.9226.1,GIZ Mgmt - Vertrag 1 - Unterstützung des BMU ...,GIZ Mgmt - Vertrag 1 - Unterstützung des BMU b...,99810.0,Nicht spezifizierte Sektoren (fortgeführt als ...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17;,EU Twinning,EU Twinning,Gestaltungsspielraum


In [51]:
# match with project number
merged_df = pd.merge(running_giz_df, bmz_df[["pn", "title_en", "sgd_pred_code"]], on="pn", how="inner")
merged_df.dropna(subset=["sgd_pred_code"], inplace=True)
print(f"{len(merged_df)}")
merged_df.head(2)

433


,pn,"Projektbezeichnung, deutsch","Projektbezeichnung, englisch",CRS-Schlüssel,CRS-Bezeichnung,CRS-Gewichtung,2. CRS-Schlüssel,2. CRS-Bezeichnung,2. CRS-Gewichtung,3. CRS-Schlüssel,...,3. CRS-Gewichtung,4. CRS-Schlüssel,4. CRS-Bezeichnung,4. CRS-Gewichtung,SDG Hauptziele,"Projektziele, deutsch","Projektziele, englisch",BMZ-Schwerpunkt,title_en,sgd_pred_code
0,2014.0968.9,Globalvorhaben Ernährungssicherung und Resilie...,"Global programme Food and Nutrition Security, ...",43072.0,Programme zur Ernährungssicherung auf Haushalt...,0.7,43071.0,Politiken/Verwaltungsführung zur Ernährungssic...,0.3,NaN,...,NaN,NaN,NaN,NaN,02;,Die Ernährungssituation und die Resilienz gege...,The food situation and the resilience to hunge...,"Sicherung der Ernährung, Landwirtschaft","Global programme Food and Nutrition Security, ...",8
1,2014.2267.4,Modernisierung kommunaler Dienstleistungen,Modernization of Local Public Services,43030.0,Stadtentwicklung und -verwaltung,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,11;,In ländlichen Gemeinden werden kommunale Diens...,Efficient and sustainable provision of local p...,"Demokratie, Zivilgesellschaft, und öffentliche...",Modernization of Local Public Services,3


In [55]:
# just use most important columns
analy_df = merged_df[["pn", "Projektbezeichnung, deutsch", "title_en", "SDG Hauptziele", "sgd_pred_code"]]
analy_df.head(400)

,pn,"Projektbezeichnung, deutsch",title_en,SDG Hauptziele,sgd_pred_code
0,2014.0968.9,Globalvorhaben Ernährungssicherung und Resilie...,"Global programme Food and Nutrition Security, ...",02;,8
1,2014.2267.4,Modernisierung kommunaler Dienstleistungen,Modernization of Local Public Services,11;,3
2,2014.2267.4,Modernisierung kommunaler Dienstleistungen,Modernization of Local Public Services,11;,3
3,2014.2456.3,Beschäftigung für nachhaltige Wirtschaftentwic...,Employment and Skills for Development in Africa,08;,14
4,2014.2456.3,Beschäftigung für nachhaltige Wirtschaftentwic...,Employment and Skills for Development in Africa,08;,14
...,...,...,...,...,...
395,2023.2084.4,Förderung von Beschäftigung in der Digitalwirt...,Promotion of Employment in the Digital Economy,08;,15
396,2023.2084.4,Förderung von Beschäftigung in der Digitalwirt...,Promotion of Employment in the Digital Economy,08;,15
397,2023.2085.1,Förderung der wirtschaftlichen Transformation,Supporting Economic Transformation (SET) Priva...,08;,14
398,2023.2085.1,Förderung der wirtschaftlichen Transformation,Supporting Economic Transformation (SET) Priva...,08;,14


In [53]:
# change dtype that both cols can be compared
analy_df["SDG Hauptziele"] = analy_df["SDG Hauptziele"].str[:2].astype(float)

analy_df.head()

C:\Users\UserNA6153\AppData\Local\Temp\ipykernel_15396\3533025184.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analy_df["SDG Hauptziele"] = analy_df["SDG Hauptziele"].str[:2].astype(float)


,pn,"Projektbezeichnung, deutsch",SDG Hauptziele,sgd_pred_code
0,2014.0968.9,Globalvorhaben Ernährungssicherung und Resilie...,2.0,8
1,2014.2267.4,Modernisierung kommunaler Dienstleistungen,11.0,3
2,2014.2267.4,Modernisierung kommunaler Dienstleistungen,11.0,3
3,2014.2456.3,Beschäftigung für nachhaltige Wirtschaftentwic...,8.0,14
4,2014.2456.3,Beschäftigung für nachhaltige Wirtschaftentwic...,8.0,14


## **Analysis**

### Compare ground truth with predicted

In [54]:
cm = confusion_matrix(analy_df['SDG Hauptziele'], analy_df['sgd_pred_code'])
print(cm)

[[ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  4  1  0  0  0  0  4  0  0]
 [ 0  0  0  0  0  0  0  0 16  0  0  0  0  0  0  0]
 [ 4  0  0  0  5  0  0  0  0 30  4  0  6 27 14  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  2  0  9  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0]
 [ 0  1  5  9  2  0  0  0  1  0  5  0  1 76 27  0]
 [ 0  0  1  1  1  0  0  0  0  1  0  2  2  0  7  0]
 [ 1  0  0  0  0  0  0  0  1  0  0  0  0 11  2  0]
 [ 0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 13  1  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  5  0  0  0  0  1  0  0  0  0]
 [ 9  4 45  2  5  0  1  0  5  0 11  0  0 11 25  0]]
